<a href="https://colab.research.google.com/github/bforseth15/WSU-College-Portfolio/blob/main/RetirementFundzs_MasterDoc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Creating a Retirement App Using Python
## FIN 429 - Financial Modeling

In [1]:
# First, the notebook must install Streamlit (necessary for Web Apps)
!pip install streamlit pyngrok -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 63.1 MB/s eta 0:00:00


### Application Script (app.py):

**Page Setup:** The script starts with setting up the Streamlit page, including the title ("Financial Planning Calculator").

**User Inputs:** The app collects various user inputs:

**Monthly Income:** User inputs for annual salary and tax rate.

**Monthly Expenses:** Inputs for monthly rental, food budget, unforeseen expenses, transport fees, utilities, and entertainment budget.

### Calculations for net monthly take-home salary and total monthly expenses:
**Savings Forecast:**

Inputs for forecasting years. Calculations to forecast future salary, expenses, and savings over the given period, consider adding inflation and salary growth.

**Visualization:**

A Plotly graph visualizing the forecasted salary, expenses, and cumulative savings over the years.

In [3]:
# This text allows a Jupyter Notebook to interact like a local .py file
%%writefile app.py

# These three bring in streamlit, plotting software, and our favorite - numpy
import streamlit as st
import plotly.graph_objects as go
import numpy as np

st.set_page_config(page_title="Simple Financial Planner")

st.title("Simple Financial Planner")
st.subheader("Plan your retirement savings with a basic income and expense tracker.")

# Section 1: Monthly Income
st.header("Monthly Income")
annual_salary = st.number_input("Enter your annual salary ($):", min_value=0.0, step=1000.0, value=50000.0)
years_to_retirement = st.slider("Enter how years until retirement:", min_value=1, max_value=50, value=30, step=1)
growth = st.slider("Expected Annual Salary Growth per year (%):", min_value=0.0, max_value=20.0, value=10.0) / 100
growth_years = st.slider("How many years between pay raises (%):", min_value=0.0, max_value=10.0, value=5.0)
tax_rate = st.slider("Select your tax rate (%):", min_value=0.0, max_value=50.0, value=20.0, step=1.0)

# Calculate take-home monthly income for first year
monthly_income1 = (annual_salary * (1 - tax_rate / 100)) / 12
st.write(f"Your monthly take-home income for this year is: **${monthly_income1:.2f}**")

# Calculate average take-home monthly income for any number of years including salary growth
monthly_income2 = (((annual_salary * years_to_retirement)* ((1 + (growth)) ** years_to_retirement)) * (1 - tax_rate / 100) / years_to_retirement) / 12
st.write(f"Your average monthly take-home income until retirement factoring in salary growth: **${monthly_income2:.2f}**")

# Section 2: Monthly Expenses
st.header("Monthly Expenses")
rent = st.number_input("Monthly Rent ($):", min_value=0.0, step=50.0, value=1000.0)
food = st.number_input("Monthly Food Budget ($):", min_value=0.0, step=50.0, value=500.0)
transport = st.number_input("Monthly Transport Costs ($):", min_value=0.0, step=50.0, value=200.0)

total_expenses = rent + food + transport
monthly_savings = max(monthly_income2 - total_expenses, 0)

st.write(f"Your total monthly expenses are: **${total_expenses:.2f}**")
st.write(f"Your estimated monthly savings are: **${monthly_savings:.2f}**")

 #loan Repayment and Visualization
st.header('Loan Repayment')
loan_value = st.number_input("Enter the loan value:", min_value=0.0, step=1000.0, value=25000.0)
loan_interest_rate = st.slider("Select the loan interest rate (%):", min_value=0.0, max_value=50.0, value=10.0, step=.5)
loan_true_interest_rate = loan_interest_rate *.01
loan_payment_frequency = st.slider("Select the loan payment frequency (months):", min_value=1, max_value=24, value=12, step=1)
loan_term = st.slider("Select the loan term (years):", min_value=1, max_value=30, value=10, step=1)
total_months2 = loan_term * 12
loan_total_payment = loan_value * (1 + loan_interest_rate / 100 / loan_payment_frequency) ** (loan_payment_frequency * loan_term)

monthly_payment = st.slider("Monthly payment (Default is 20% of monthly savings) ($):", min_value=0.0, step=50.0, max_value= monthly_savings, value=monthly_savings*.2)


# Loan Repayment Simulation
st.subheader("Loan Balance Over Time")

loan_balance = loan_value
loan_over_time = []
months = 0

# Avoid infinite loop by checking if payment is sufficient
if monthly_payment <= (loan_balance * (loan_true_interest_rate / 12)):
    st.write("⚠️ **Warning:** Monthly payment is too low! Interest will keep increasing the balance.")
else:
    # Loan amortization calculation
    while loan_balance > 0:
        interest_for_month = loan_balance * (loan_true_interest_rate / 12)  # Monthly interest
        principal_paid = max(0, monthly_payment - interest_for_month)  # Principal portion
        loan_balance = max(0, loan_balance - principal_paid)  # Update balance

        loan_over_time.append(loan_balance)
        months += 1

        # Stop early if rounding errors keep balance slightly above 0
        if loan_balance < 0.01:
            loan_balance = 0
            break

    # Plot Loan Balance Over Time
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=list(range(months)), y=loan_over_time, mode='lines', name='Loan Balance'))
    fig.update_layout(title="Loan Balance Over Time", xaxis_title="Months", yaxis_title="Remaining Loan ($)")
    st.plotly_chart(fig, use_container_width=True)

    st.write(f"Loan will be paid off in **{months} months** ({months//12} years, {months%12} months).")
    st.write(f"Totaling **${monthly_payment * months:.2f}** across the life of the loan")
    st.write(f"Your new monthly savings after loan payments is **${monthly_savings - monthly_payment:.2f}**")

    monthly_savings_2 = monthly_savings - monthly_payment

# Section 3: Simple Simulation
st.header("Retirement Savings Simulation")
st.subheader("Estimate your retirement savings based on monthly contributions and expected returns.")

current_savings = st.number_input("Current Savings ($):", min_value=0.0, step=1000.0, value=10000.0)
monthly_contribution = st.slider(
    "Set your monthly contribution to savings ($):",
    min_value=0.0,
    max_value=monthly_savings_2,
    value=min(500.0, monthly_savings),
    step=50.0
)
monthly_savings_3 = monthly_savings_2 - monthly_contribution
annual_return = st.slider("Expected Annual Return on Investments (%):", min_value=0.0, max_value=15.0, value=8.0) / 100

# Projected savings calculation
total_months = years_to_retirement * 12
monthly_return = (1 + annual_return) ** (1 / 12) - 1
future_savings = current_savings * (1 + monthly_return) ** total_months
for month in range(total_months):
    future_savings += monthly_contribution * (1 + monthly_return) ** (total_months - month)

st.write(f"Projected savings at retirement: **${future_savings:,.2f}**")

# Visualization
st.subheader("Savings Over Time")
savings_over_time = [current_savings]
for month in range(1, total_months + 1):
    savings_over_time.append(
        savings_over_time[-1] * (1 + monthly_return) + monthly_contribution
    )
fig = go.Figure()
fig.add_trace(go.Scatter(x=list(range(total_months + 1)), y=savings_over_time, mode='lines', name='Savings'))
fig.update_layout(title="Projected Savings Over Time", xaxis_title="Months", yaxis_title="Total Savings ($)")
st.plotly_chart(fig, use_container_width=True)

max_savings_value = monthly_savings_2 - monthly_contribution

# Simple Savings Calculator
# Header
st.header("Simple Savings Calculator")
st.subheader("Add in your monthly contribution, expected annual interest rate, and length of saving. This will return an calculated Account Balance.")
# What are you saving for
saving_name = st.text_input("What are you saving for? ")
# what is your montly contribution
sscmonth_contribution = st.slider("What is your monthly contribution? ",
                               min_value=0.0,
                               max_value=max_savings_value,
                               value=10.0,
                               step=.5)
# What is your annual expected return rate
annual_err = st.slider("Select the Annual Expected Return Rate (%):", min_value=0.0, max_value=50.0, value=10.0, step=.5)
# When do expect to use the money
savings_length = st.slider("How long are you planning to save (years) ?:", min_value=1, max_value=30, value=10, step=1)
annual_rate = annual_err / 100
# Calculation : If you save (monthly contribution) at (annual_err) for (savings_length), your account balance will be ___
SSC_calculation = sscmonth_contribution * (((1 + annual_rate / 12) ** (12 * savings_length) - 1) / ((annual_rate) / 12))

# Display the result
st.write(f"Projected savings **${SSC_calculation:,.2f}.")

def calculate_debt_reduction(current_debt, monthly_payment, interest_rate):
    debt_balance = current_debt
    debt_over_time = []
    months = 0
    interest_rate_monthly = (interest_rate / 100) / 12  # Convert annual rate to monthly rate

    if monthly_payment <= (current_debt * interest_rate_monthly):
        st.error("Monthly payment is too low to cover interest. Increase the payment amount.")
        return [], 0

    while debt_balance > 0:
        interest = debt_balance * interest_rate_monthly
        principal = monthly_payment - interest
        debt_balance -= principal
        debt_balance = max(debt_balance, 0)  # Ensure balance does not go negative
        debt_over_time.append(debt_balance)
        months += 1
        if debt_balance == 0:
            break

    return debt_over_time, months

# Debt Parameter
st.header("Debt Repayment Tracker")
st.subheader("View and Track Your Debt Reduction")

Current_Debt = st.number_input("Current Debt ($):", min_value=0.0, step=1000.0, value=10000.0)
Monthly_Payment = st.number_input("Monthly Payment ($):", min_value=0.0, step=50.0, value=200.0)
Interest_Rate = st.slider("Select the interest rate (%):", min_value=0.0, max_value=50.0, value=10.0, step=0.5)

if st.button("Calculate Repayment Plan"):
    debt_over_time, months_needed = calculate_debt_reduction(Current_Debt, Monthly_Payment, Interest_Rate)

    if debt_over_time:
        st.success(f"Debt will be fully paid off in {months_needed} months (~{months_needed // 12} years, {months_needed % 12} months).")

        # Debt Parameter Visualization
        st.subheader("Debt Tracker and Visualization")
        fig = go.Figure()
        fig.add_trace(go.Scatter(x=list(range(1, len(debt_over_time) + 1)), y=debt_over_time, mode='lines+markers', name='Debt Balance'))
        fig.update_layout(title="Debt Balance Over Time", xaxis_title="Months", yaxis_title="Remaining Debt ($)", showlegend=True)
        st.plotly_chart(fig, use_container_width=True)




Overwriting app.py


In [4]:
!ngrok authtoken 2rlxC0xxkAihRSR7Lo4gZ8DkNjv_2jNN2tAPP1UZMHL8BBPB4

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


### You shoulnd't change any of the following to run this in your browser

In [5]:
# These next three are for hosting the app
import subprocess
import time
from pyngrok import ngrok

# Kill any existing Streamlit processes
!pkill streamlit

# Start Streamlit in the background
subprocess.Popen(["streamlit", "run", "app.py", "--server.port", "8501"])

# Give Streamlit time to start
time.sleep(3)

# Open an Ngrok tunnel to the Streamlit app
public_url = ngrok.connect(8501, "http")
print(f"Streamlit App is live at: {public_url}")

Streamlit App is live at: NgrokTunnel: "https://1ca57f04a313.ngrok-free.app" -> "http://localhost:8501"


In [ ]:
# When finished, run this cell to end your ngrok session
ngrok.kill()

# Assignment:

#### Students should expand their financial model beyond the provided template. For full credit, each project must include:

1. At least three new variables (e.g., different expense categories, savings strategies, or loan repayment details).
1. 2-3 new visualizations to better illustrate financial projections.
1. A new dynamic function (e.g., a slider, interactive input, or real-time updates).

Students may modify the existing model or build their own from scratch. Completed projects will be hosted on Streamlit's Cloud.